In [ ]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir
    path.append(dir(path[0])+'/dash_app')
    __package__ = "dash_app"

In [ ]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map, map_columns
from settings import *

In [ ]:
data_path = DATA_PATH+'Atmospheric_Domain/2.5Precipitation/Figure2.9/'
data_csv = data_path + 'Figure2.9_data.csv'
"""
Tidy data for charts
"""
xls = pd.ExcelFile(
    data_path+'Annual_rainfall_totals_and_anomalies_OverIreland.xlsx')
original_df = pd.read_excel(xls, '2010SeasannRR',skiprows=[])
original_df
columns_dict = {'datetime':'years',
               'sum__annual__precipitation':'ANN',
               'normal_1961_1990__precipitation':'ANNmean',
               'moving_average_of_sum__11year__precipitation':'Unnamed: 3',
               'sum__annual__precipitation_anomaly':'Unnamed: 4',
                  }
df = map_columns(columns_dict, original_df)
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y')
df['location'] = 'Ireland'
df['moving_average_of_sum__11year__precipitation_anomaly']=df['moving_average_of_sum__11year__precipitation']-1186.913018
df['average_1990_2019__precipitation']=79.3
# df.dropna(subset=['mean__annual__surface_air_temperature'], inplace=True)
# df.to_csv(data_csv)
# df[0:20]

In [ ]:
data_path = DATA_PATH+'Atmospheric_Domain/2.5Precipitation/Figure2.9/'
data_csv = data_path + 'Figure2.9_data.csv'
df = pd.read_csv(data_csv, index_col=0)
df['datetime'] = pd.to_datetime(df['datetime'])
annualTrace = go.Bar(x=df["datetime"],
                     y=df["sum__annual__precipitation_anomaly"],
                     text=df["sum__annual__precipitation"],
                     name='Annual',
                     marker=dict(
    # color="#214a7b", color used in report
    color=TIMESERIES_COLOR_2,
    opacity=0.5
),
    hovertemplate='%{x|%Y}<br>' +
    '<b>Annual</b><br>' +
    'Total: %{text:.2f} mm<br>' +
    'Anomaly: %{y:.2f} mm<extra></extra>'
)
movingAverage = go.Scatter(x=df["datetime"],
                           y=df["moving_average_of_sum__11year__precipitation_anomaly"],
                           text=df["moving_average_of_sum__11year__precipitation"],
                           name='11 Year Moving Average',
                           mode='lines',  # 'line' is default
                           line_shape='spline',
                           line=dict(
    # color="#fc0d1b", color used in report
    color=TIMESERIES_COLOR_1,
    width=2),
    hovertemplate='%{x|%Y}<br>' +
    '<b>11 Year Moving Average</b><br>' +
    'Total: %{text:.2f} mm<br>' +
    'Anomaly: %{y:.2f} mm<extra></extra>'
)
normal = go.Scatter(x=df["datetime"],
                    y=df["normal_1961_1990__precipitation"],
                    name='1961-1990 Normal',
                    mode='lines',  # 'line' is default
                    line_shape='spline',
                    line=dict(color=TIMESERIES_COLOR_3, 
                              width=2),
                    hoverinfo='skip',
                    )
average1990_2019 = go.Scatter(x=df["datetime"],
                              y=df["average_1990_2019__precipitation"],
                              name='1990-2019 Average',
                              mode='lines',  # 'line' is default
                              line_shape='spline',
                              line=dict(color=TIMESERIES_COLOR_3, #""#22b2ed",  # color used in report
                                        dash='dash',
                                        width=2),
                              hoverinfo='skip',
                              )
figure_2_9 = make_subplots(specs=[[{'secondary_y': True}]])
figure_2_9.add_trace(annualTrace,
                     secondary_y=False,)
figure_2_9.add_trace(movingAverage,
                     secondary_y=False,)
figure_2_9.add_trace(normal,
                     secondary_y=True,)

figure_2_9.add_trace(average1990_2019,
                     secondary_y=False,)

figure_2_9.update_layout(TIMESERIES_LAYOUT)
figure_2_9.update_yaxes(title_text='Difference (mm) from 1961-1990 Normal',
                        secondary_y=False,
                        range=[-300, 330],
                        showgrid=False,
                        dtick=100,  # dtick sets the distance between ticks
                        tick0=0,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        showspikes=True,
                        # zeroline=True,  # add a zero line
                        # zerolinecolor=TIMESERIES_COLOR_2
                        )
figure_2_9.update_yaxes(title_text='Annual Rainfall Total (mm)',
                        secondary_y=True,
                        range=[886, 1517],
                        showgrid=False,
                        dtick=100,  # dtick sets the distance between ticks
                        tick0=1186,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        )

figure_2_9.update_xaxes(
    title='Year',
    fixedrange=True,
#     tickformat='000',
    showspikes=True,
    spikethickness=2,
)
figure_2_9

In [ ]:
"""
Read from Excel, No longer used
"""

data_path = DATA_PATH+'Atmospheric_Domain/2.5Precipitation/Figure2.9/'
xls = pd.ExcelFile(
    data_path+'Annual_rainfall_totals_and_anomalies_OverIreland.xlsx')
dataDF = pd.read_excel(xls, '2010SeasannRR')
dataDF.rename(columns = {
    "Unnamed: 3":"11 Year Moving Average Totals",
    "Unnamed: 4":"Anomaly",
    "11 year moving average":"11 Year Moving Average Anomaly"
    }, inplace = True)
movingAverageTotals = dataDF.ANN.rolling(window=11, center=True).mean() 
dataDF["11 Year Average"]=movingAverageTotals
annualTrace = go.Bar(x=dataDF["years"],
                     y=dataDF["Anomaly"],
                     text=dataDF["ANN"],
                     name='Annual',
                     marker=dict(
    # color="#214a7b", color used in report
    color=TIMESERIES_COLOR_2,
    opacity=0.5
),
    hovertemplate='%{x}<br>' +
    '<b>Annual</b><br>' +
    'Total: %{text:.2f} mm<br>' +
    'Anomaly: %{y:.2f} mm<extra></extra>'
)
movingAverage = go.Scatter(x=dataDF["years"],
                           y=dataDF["11 Year Moving Average Anomaly"],
                           text=dataDF["11 Year Moving Average Totals"],
                           name='11yr Moving Average',
                           mode='lines',  # 'line' is default
                           line_shape='spline',
                           line=dict(
    # color="#fc0d1b", color used in report
    color=TIMESERIES_COLOR_1,
    width=2),
    hovertemplate='%{x}<br>' +
    '<b>11yr Moving Average</b><br>' +
    'Total: %{text:.2f} mm<br>' +
    'Anomaly: %{y:.2f} mm<extra></extra>'
)
normal = go.Scatter(x=dataDF["years"],
                    y=dataDF["ANNmean"],
                    name='1961-1990 Normal',
                    mode='lines',  # 'line' is default
                    line_shape='spline',
                    line=dict(color=TIMESERIES_COLOR_3, 
                              width=2),
                    hoverinfo='skip',
                    )
average1990_2019 = go.Scatter(x=dataDF["years"],
                              y=dataDF["1990-2019 Average"],
                              name='1990-2019 Average',
                              mode='lines',  # 'line' is default
                              line_shape='spline',
                              line=dict(color=TIMESERIES_COLOR_3, #""#22b2ed",  # color used in report
                                        dash='dash',
                                        width=2),
                              hoverinfo='skip',
                              )
figure_2_9 = make_subplots(specs=[[{'secondary_y': True}]])
figure_2_9.add_trace(annualTrace,
                     secondary_y=False,)
figure_2_9.add_trace(movingAverage,
                     secondary_y=False,)
figure_2_9.add_trace(normal,
                     secondary_y=True,)

figure_2_9.add_trace(average1990_2019,
                     secondary_y=False,)

figure_2_9.update_layout(TIMESERIES_LAYOUT)
figure_2_9.update_yaxes(title_text='Difference (mm) from 1961-1990 Normal',
                        secondary_y=False,
                        range=[-300, 330],
                        showgrid=False,
                        dtick=100,  # dtick sets the distance between ticks
                        tick0=0,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        showspikes=True,
                        # zeroline=True,  # add a zero line
                        # zerolinecolor=TIMESERIES_COLOR_2
                        )
figure_2_9.update_yaxes(title_text='Annual Rainfall Total (mm)',
                        secondary_y=True,
                        range=[886, 1517],
                        showgrid=False,
                        dtick=100,  # dtick sets the distance between ticks
                        tick0=1186,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        )

figure_2_9.update_xaxes(
    title='Year',
    fixedrange=True,
    tickformat='000',
    showspikes=True,
    spikethickness=2,
)
figure_2_9

In [ ]:
# used for static images on main site
figure_2_9.update_layout(
    margin={"t": 40, "b": 20, "r": 40, "l": 80, },
    plot_bgcolor='#f7fbfd',
    paper_bgcolor='white',
    height=500,
    width=900,
    font_size=14,
    title=dict(
        text='<b>Mean Annual Precipitation - Ireland (1941-2019)</b>',
        x=0.5,
        y=0.97),)
figure_2_9.update_yaxes(
    secondary_y=False,
    title_standoff=0.2
)
figure_2_9.update_xaxes(
    title_standoff=1
)
figure_2_9.write_image(
    "/Users/dan/ClimateIreland/CI-Status-Report/images/mean__annual__precipitation.png", 
    format="png"
    )

In [ ]:
data_path = DATA_PATH+'Atmospheric_Domain/2.5Precipitation/Figure2.10/'
df = pd.read_csv(data_path+'Fig2.10_StationsTable.txt', delimiter = ",")
df=df.round(2) 
df


In [ ]:
cwd1DF = df.loc[df['CWD_decade']<= 0]
cwd1DFStr=cwd1DF.astype(str)
cwd1Trend = go.Scattermapbox(
        name='<= 0.0',
        lon=cwd1DF.lon,
        lat=cwd1DF.lat,
        marker=dict(color="#4ce600",#green
                    size=8,),
        hovertemplate='<b>'+cwd1DF["station_id"]+'</b><br>' +
       'Trend in CWD per Decade: ' + cwd1DFStr["CWD_decade"]+ ' days<br>' +
        'Trend in CWD per Year: ' + cwd1DFStr["CWD_year"]+' days<br>' +
        'Lat: ' + cwd1DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + cwd1DFStr["lon"]+ '\u00b0<br>' +
        '<extra><br>'+
        'Trend in CDD per Decade: ' + cwd1DFStr["CDD_decade"]+' days<br>' +
        'Trend in CDD per Year: ' + cwd1DFStr["CDD_year"]+' days<br>' +
        '</extra>',
    )

cwd2DF = df.loc[(df['CWD_decade']>0)&(df['CWD_decade']<=0.5)]
cwd2DFStr=cwd2DF.astype(str)
cwd2Trend = go.Scattermapbox(
        name='> 0.0, <= 0.5',
        lon=cwd2DF.lon,
        lat=cwd2DF.lat,
        marker=dict(color="#004da8",#blue
                    size=10,),
        hovertemplate='<b>'+cwd2DFStr["station_id"]+'</b><br>' +
       'Trend in CWD per Decade: ' + cwd2DFStr["CWD_decade"]+' days<br>' +
        'Trend in CWD per Year: ' + cwd2DFStr["CWD_year"]+' days<br>' +
        'Lat: ' + cwd2DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + cwd2DFStr["lon"]+ '\u00b0<br>' +
        'Trend in CDD per Decade: ' + cwd2DFStr["CDD_decade"]+' days<br>' +
        'Trend in CDD per Year: ' + cwd2DFStr["CDD_year"]+' days<br>' +
        '<extra></extra>',
    )
cwd3DF = df.loc[(df['CWD_decade']>0.5)&(df['CWD_decade']<=1)]
cwd3DFStr=cwd3DF.astype(str)
cwd3Trend = go.Scattermapbox(
        name='> 0.5, <= 1.0',
        lon=cwd3DF.lon,
        lat=cwd3DF.lat,
        marker=dict(color="#a900e6",#pink
                    size=12,),
        hovertemplate='<b>'+cwd3DFStr["station_id"]+'</b><br>' +
       'Trend in CWD per Decade: ' + cwd3DFStr["CWD_decade"]+' days<br>' +
        'Trend in CWD per Year: ' + cwd3DFStr["CWD_year"]+' days<br>' +
        'Lat: ' + cwd3DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + cwd3DFStr["lon"]+ '\u00b0<br>' +
        'Trend in CDD per Decade: ' + cwd3DFStr["CDD_decade"]+' days<br>' +
        'Trend in CDD per Year: ' + cwd3DFStr["CDD_year"]+' days<br>' +
        '<extra></extra>',
    )
cwd4DF = df.loc[df['CWD_decade']>1.0]
cwd4DFStr=cwd4DF.astype(str)
cwd4Trend = go.Scattermapbox(
        name='> 1.0',
        lon=cwd4DF.lon,
        lat=cwd4DF.lat,
        marker=dict(color="#4c0073",#purple
                    size=14,),
        hovertemplate='<b>'+cwd4DFStr["station_id"]+'</b><br>' +
       'Trend in CWD per Decade: ' + cwd4DFStr["CWD_decade"]+' days<br>' +
        'Trend in CWD per Year: ' + cwd4DFStr["CWD_year"]+' days<br>' +
        'Lat: ' + cwd4DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + cwd4DFStr["lon"]+ '\u00b0<br>' +
        'Trend in CDD per Decade: ' + cwd4DFStr["CDD_decade"]+' days<br>' +
        'Trend in CDD per Year: ' + cwd4DFStr["CDD_year"]+' days<br>' +
        '<extra></extra>',
    )

In [ ]:
figure_2_10_1 = go.Figure(
        data=[cwd1Trend,cwd2Trend,cwd3Trend,cwd4Trend],
        layout=MAP_LAYOUT)
figure_2_10_1.update_layout(legend_title="<b>Trend in maximum length of"+
                      "<br>annual wet spell days (CWD) per decade</b>")
figure_2_10_1

In [ ]:
cdd1DF = df.loc[df['CDD_decade']<= 0]
cdd1DFStr=cdd1DF.astype(str)
cdd1Trend = go.Scattermapbox(
        name='<= 0.0',
        lon=cdd1DF.lon,
        lat=cdd1DF.lat,
        marker=dict(color="#70a800",#green
                    size=8,),
        hovertemplate='<b>'+cdd1DF["station_id"]+'</b><br>' +
       'Trend in CDD per Decade: ' + cdd1DFStr["CDD_decade"]+ ' days<br>' +
        'Trend in CDD per Year: ' + cdd1DFStr["CDD_year"]+' days<br>' +
        'Lat: ' + cdd1DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + cdd1DFStr["lon"]+ '\u00b0<br>' +
        '<extra><br>'+
        'Trend in CWD per Decade: ' + cdd1DFStr["CWD_decade"]+' days<br>' +
        'Trend in CWD per Year: ' + cdd1DFStr["CWD_year"]+' days<br>' +
        '</extra>',
    )

cdd2DF = df.loc[(df['CDD_decade']>0)&(df['CDD_decade']<=0.5)]
cdd2DFStr=cdd2DF.astype(str)
cdd2Trend = go.Scattermapbox(
        name='> 0.0, <= 0.5',
        lon=cdd2DF.lon,
        lat=cdd2DF.lat,
        marker=dict(color="#ffff00",#yellow
                    size=10,),
        hovertemplate='<b>'+cdd2DFStr["station_id"]+'</b><br>' +
       'Trend in CDD per Decade: ' + cdd2DFStr["CDD_decade"]+' days<br>' +
        'Trend in CDD per Year: ' + cdd2DFStr["CDD_year"]+' days<br>' +
        'Lat: ' + cdd2DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + cdd2DFStr["lon"]+ '\u00b0<br>' +
        'Trend in CWD per Decade: ' + cdd2DFStr["CWD_decade"]+' days<br>' +
        'Trend in CWD per Year: ' + cdd2DFStr["CWD_year"]+' days<br>' +
        '<extra></extra>',
    )
cdd3DF = df.loc[(df['CDD_decade']>0.5)&(df['CDD_decade']<=1)]
cdd3DFStr=cdd3DF.astype(str)
cdd3Trend = go.Scattermapbox(
        name='> 0.5, <= 1.0',
        lon=cdd3DF.lon,
        lat=cdd3DF.lat,
        marker=dict(color="#e69800",#orange
                    size=12,),
        hovertemplate='<b>'+cdd3DFStr["station_id"]+'</b><br>' +
       'Trend in CDD per Decade: ' + cdd3DFStr["CDD_decade"]+' days<br>' +
        'Trend in CDD per Year: ' + cdd3DFStr["CDD_year"]+' days<br>' +
        'Lat: ' + cdd3DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + cdd3DFStr["lon"]+ '\u00b0<br>' +
        'Trend in CWD per Decade: ' + cdd3DFStr["CWD_decade"]+' days<br>' +
        'Trend in CWD per Year: ' + cdd3DFStr["CWD_year"]+' days<br>' +
        '<extra></extra>',
    )
cdd4DF = df.loc[df['CDD_decade']>1.0]
cdd4DFStr=cdd4DF.astype(str)
cdd4Trend = go.Scattermapbox(
        name='> 1.0',
        lon=cdd4DF.lon,
        lat=cdd4DF.lat,
        marker=dict(color="#e60000",#red
                    size=14,),
        hovertemplate='<b>'+cdd4DFStr["station_id"]+'</b><br>' +
       'Trend in CDD per Decade: ' + cdd4DFStr["CDD_decade"]+' days<br>' +
        'Trend in CDD per Year: ' + cdd4DFStr["CDD_year"]+' days<br>' +
        'Lat: ' + cdd4DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + cdd4DFStr["lon"]+ '\u00b0<br>' +
        'Trend in CWD per Decade: ' + cdd4DFStr["CWD_decade"]+' days<br>' +
        'Trend in CWD per Year: ' + cdd4DFStr["CWD_year"]+' days<br>' +
        '<extra></extra>',
    )

In [ ]:
figure_2_10_2 = go.Figure(
        data=[cdd1Trend,cdd2Trend,cdd3Trend,cdd4Trend],
        layout=MAP_LAYOUT)
figure_2_10_2.update_layout(legend_title="<b>Trend in maximum length of"+
                      "<br>annual dry spell days (CDD) per decade</b>")
figure_2_10_2

In [ ]:

df.loc[df['CWD_decade']<=0,'group']=1
df.loc[(df['CWD_decade']>0)&(df['CWD_decade']<=0.5),'group']=2
df.loc[(df['CWD_decade']>0.5)&(df['CWD_decade']<1),'group']=3
df.loc[df['CWD_decade']>=1.0,'group']=4
df

In [ ]:
# colors = ["purple","pink","blue","green"]
colors=["#4c0073","#a900e6","#004da8","#4ce600"]
wetDayTrend = go.Scattermapbox(
        name='Wet Days',
        lon=df.lon,
        lat=df.lat,
        text=df['CWD_decade'],
        marker=dict(color=df['group'],
                    colorscale=colors,
                    size=df['group']*8,
                    reversescale=True,
                    showscale=False,
                    colorbar=dict(title="Days")),
        hovertemplate=df.station_id +
        '<br>Lat: %{lon:.2f}\u00b0<br>' +
        'Lon: %{lat:.2f}\u00b0<br>' +
        'CWD: %{text:.1f} days<br>' +
        '<extra></extra>',
    )
map_2_5 = go.Figure(
        data=[wetDayTrend],
        layout=MAP_LAYOUT)
map_2_5.update_layout(
        autosize=True,
        xaxis={'title': 'x-axis','fixedrange':True},
        yaxis={'title': 'y-axis','fixedrange':True},
        mapbox=dict(bearing=0,
                center=dict(
                    lat=53.5,
                    lon=352
                ),
                pitch=0,
                zoom=5.2,)
)
map_2_5.add_shape(type="circle",
    xref="paper", yref="paper",
    x0=0.1, y0=0.8, x1=0.2, y1=1,
    line_color="LightSeaGreen",
)
map_2_5.add_annotation(
    xref="paper", yref="paper",
    x=0.1,
    y=0.8,
    showarrow=False,
    text="An annotation whose text and arrowhead reference the axes and the data",
)

map_2_5

In [ ]:
data_path = DATA_PATH+'Atmospheric_Domain/2.5Precipitation/Map2.5/'
dataDF = pd.read_csv(data_path+'Map2.5_StationTable.txt', delimiter = ",")
dataDF

In [ ]:
data_path = DATA_PATH+'Atmospheric_Domain/2.5Precipitation/Map2.5/'
dataDF = pd.read_csv(data_path+'Map2.5_StationTable.txt', delimiter = ",")
map_2_5=stations_map(dataDF)
map_2_5